<h1>API Valencia polución</h1>

url: https://valencia.opendatasoft.com/explore/dataset/estacions-contaminacio-atmosferiques-estaciones-contaminacion-atmosfericas/information/?location=13,39.46576,-0.36855&basemap=e4bf90

In [7]:
from time import sleep
from os import path

import requests
import datetime
import pandas as pd

In [8]:
def make_request(base_url, params):
    response = requests.get(base_url, params=params)
    if response:
        return response.json()
    else:
        raise Exception("Error Downloading JSON")

In [9]:
base_url = "https://valencia.opendatasoft.com/api/explore/v2.1/catalog/datasets/estacions-contaminacio-atmosferiques-estaciones-contaminacion-atmosfericas/records"

In [10]:
params = {'limit': 0}
n_stations = make_request(base_url, params)['total_count']
print(f"Total Number of Active Stations = {n_stations}")

Total Number of Active Stations = 11


In [11]:
first_time = True
max_records_per_request = 100

current_time = 0
sleep_time = 60 * 60
total_time = (24 * 60 * 60) + sleep_time
#total_time = 1  # Esto parece forzar una única iteración, útil para pruebas

path_csv = ['res', 'valencia_pollution_dataset.csv']
path_csv_solved = path.join(*path_csv)

In [ ]:
while current_time < total_time:
    offset = 0
    pollution_list = []

    while offset < n_stations:
        params = {
            "limit": max_records_per_request,
            "offset": offset
        }
        sub_list = make_request(base_url, params)['results']
        pollution_list += sub_list
        offset += max_records_per_request

    print(f"Current Time = {current_time}, Records Processed = {len(pollution_list)}, fecha y hora: {datetime.datetime.now()}")

    df = pd.DataFrame(pollution_list, columns=[
        'objectid', 'nombre', 'direccion', 'tipozona', 'tipoemisio',
        'so2', 'no2', 'o3', 'co', 'pm10', 'pm25', 'fecha_carg',
        'calidad_am'
    ])

    df.to_csv(
        path_csv_solved,
        sep=';',
        header=True if first_time else False,
        index=False,
        mode='a',
        na_rep='N/A'
    )

    first_time = False
    sleep(sleep_time)
    current_time += sleep_time

Current Time = 0, Records Processed = 11, fecha y hora: 2025-06-13 09:38:20.547111
